In [1]:
import montlake


In [2]:
help(montlake)

Help on package montlake:

NAME
    montlake

PACKAGE CONTENTS
    None
    _nbdev
    atomgeom (package)
    core
    exec (package)
    geometry (package)
    gradients (package)
    mflasso (package)
    optimization (package)
    plotting (package)
    simulations (package)
    statistics (package)
    tslasso (package)
    utils (package)
    vendor (package)

VERSION
    0.0.1

FILE
    /Users/samsonkoelle/miniconda3/envs/montlake_xxx/lib/python3.6/site-packages/montlake/__init__.py




In [4]:
# AUTOGENERATED! DO NOT EDIT! File to edit: nbs/optimization.gradientgrouplasso.ipynb (unless otherwise specified).

__all__ = ['GradientGroupLasso', 'get_sr_lambda_parallel', 'batch_stream']

# Cell
#loosely inspired by the pyglmnet package
from einops import rearrange
#import autograd.numpy as np
import numpy as np

class GradientGroupLasso:

    def __init__(self, dg_M, df_M, reg_l1s, reg_l2, max_iter,learning_rate, tol, beta0_npm= None):

        n = dg_M.shape[0]
        d= dg_M.shape[1]
        m = df_M.shape[2]
        p = dg_M.shape[2]
        dummy_beta = np.ones((n,p,m))

        self.dg_M = dg_M
        self.df_M = df_M
        self.reg_l1s = reg_l1s
        self.reg_l2 = reg_l2
        self.beta0_npm = beta0_npm
        self.n = n
        self.p = p
        self.m = m
        self.d = d
        self.dummy_beta = dummy_beta
        #self.group = np.asarray(group)
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.tol = tol
        self.Tau = None
        self.alpha = 1.
        self.lossresults = {}
        self.dls = {}
        self.l2loss = {}
        self.penalty = {}

    def _prox(self,beta_npm, thresh):
        """Proximal operator."""

        p = self.p
        result = np.zeros(beta_npm.shape)
        result = np.asarray(result, dtype = float)
        for j in range(p):
            if np.linalg.norm(beta_npm[:,j,:]) > 0.:
                potentialoutput = beta_npm[:,j,:] - (thresh / np.linalg.norm(beta_npm[:,j,:])) * beta_npm[:,j,:]
                posind = np.asarray(np.where(beta_npm[:,j,:] > 0.))
                negind = np.asarray(np.where(beta_npm[:,j,:] < 0.))
                po = beta_npm[:,j,:].copy()
                po[posind[0],posind[1]] = np.asarray(np.clip(potentialoutput[posind[0],posind[1]],a_min = 0., a_max = 1e15), dtype = float)
                po[negind[0],negind[1]] = np.asarray(np.clip(potentialoutput[negind[0],negind[1]],a_min = -1e15, a_max = 0.), dtype = float)
                result[:,j,:] = po
        return result

    def _grad_L2loss(self, beta_npm):

        df_M = self.df_M
        dg_M = self.dg_M
        reg_l2 = self.reg_l2
        dummy_beta = self.dummy_beta

        df_M_hat = np.einsum('ndp,npm->ndm',dg_M, beta_npm)
        error = df_M_hat - df_M
        grad_beta = np.einsum('ndm,ndp->npm',error,dg_M) #+ reg_l2 * np.ones()
        #if
        return grad_beta

    def _L1penalty(self, beta_npm):

        p = self.p
        m = self.m
        n = self.n
        beta_mn_p = rearrange(beta_npm, 'n p m -> (m n) p')#np.reshape(beta_mnp, ((m*n,p)))
        L1penalty = np.linalg.norm(beta_mn_p, axis = 0).sum()

        return L1penalty

    def _loss(self,beta_npm, reg_lambda):
        """Define the objective function for elastic net."""
        L = self._logL(beta_npm)
        P = self._L1penalty(beta_npm)
        J = -L + reg_lambda * P
        return J

    def _logL(self,beta_npm):

        df_M = self.df_M
        dg_M = self.dg_M

        df_M_hat = np.einsum('ndp,npm -> ndm',dg_M, beta_npm)
        logL = -0.5 * np.linalg.norm((df_M - df_M_hat))**2
        return(logL)

    def _L2loss(self,beta_npm):
        output = -self._logL(beta_npm)
        return(output)

    def fhatlambda(self,learning_rate,beta_npm_new,beta_npm_old):

        #print('lr',learning_rate)
        output = self._L2loss(beta_npm_old) + np.einsum('npm,npm', self._grad_L2loss(beta_npm_old),(beta_npm_new-beta_npm_old)) + (1/(2*learning_rate)) * np.linalg.norm(beta_npm_new-beta_npm_old)**2

        return(output)

    def _btalgorithm(self,beta_npm ,learning_rate,b,maxiter_bt,rl):

        grad_beta = self._grad_L2loss(beta_npm = beta_npm)
        for i in range(maxiter_bt):
            beta_npm_postgrad = beta_npm - learning_rate * grad_beta
            beta_npm_postgrad_postprox = self._prox(beta_npm_postgrad, learning_rate * rl)
            fz = self._L2loss(beta_npm_postgrad_postprox)
            #fhatz = self.fhatlambda(lam,beta_npm_postgrad_postprox, beta_npm_postgrad)
            fhatz = self.fhatlambda(learning_rate,beta_npm_postgrad_postprox, beta_npm)
            if fz <= fhatz:
                #print(i)
                break
            learning_rate = b*learning_rate

        return(beta_npm_postgrad_postprox,learning_rate)

    def fit(self, beta0_npm = None):

        reg_l1s = self.reg_l1s
        n = self.n
        m = self.m
        p = self.p

        dg_M = self.dg_M
        df_M = self.df_M

        tol = self.tol
        np.random.RandomState(0)

        if beta0_npm is None:
            beta_npm_hat = 1 / (n*m*p) * np.random.normal(0.0, 1.0, [n, p,m])
            #1 / (n_features) * np.random.normal(0.0, 1.0, [n_features, n_classes])
        else:
            beta_npm_hat = beta0_npm

        fit_params = list()
        for l, rl in enumerate(reg_l1s):
            fit_params.append({'beta': beta_npm_hat})
            if l == 0:
                fit_params[-1]['beta'] = beta_npm_hat
            else:
                fit_params[-1]['beta'] = fit_params[-2]['beta']

            alpha = 1.
            beta_npm_hat = fit_params[-1]['beta']
            #g = np.zeros([n_features, n_classes])
            L, DL ,L2,PEN = list(), list() , list(), list()
            learning_rate = self.learning_rate
            beta_npm_hat_1 = beta_npm_hat.copy()
            beta_npm_hat_2 = beta_npm_hat.copy()
            for t in range(0, self.max_iter):
                #print(t,l,rl)
                #print(t)
                L.append(self._loss(beta_npm_hat, rl))
                L2.append(self._L2loss(beta_npm_hat))
                PEN.append(self._L1penalty(beta_npm_hat))
                w = (t / (t+ 3))
                beta_npm_hat_momentumguess = beta_npm_hat + w*(beta_npm_hat_1 - beta_npm_hat_2)

                beta_npm_hat , learning_rate = self._btalgorithm(beta_npm_hat_momentumguess,learning_rate,.5,1000, rl)
                #print(beta_npm_hat_momentumguess.max(), beta_npm_hat.max(),self._L2loss(beta_npm_hat), learning_rate)
                beta_npm_hat_2 = beta_npm_hat_1.copy()
                beta_npm_hat_1 = beta_npm_hat.copy()

                if t > 1:
                    DL.append(L[-1] - L[-2])
                    if np.abs(DL[-1] / L[-1]) < tol:
                        print('converged', rl)
                        msg = ('\tConverged. Loss function:'
                               ' {0:.2f}').format(L[-1])
                        msg = ('\tdL/L: {0:.6f}\n'.format(DL[-1] / L[-1]))
                        break

            fit_params[-1]['beta'] = beta_npm_hat
            self.lossresults[rl] = L
            self.l2loss[rl] = L2
            self.penalty[rl] = PEN
            self.dls[rl] = DL

        self.fit_ = fit_params
        #self.ynull_ = np.mean(y)

        return self

from einops import rearrange
import numpy as np

In [6]:
import numpy as np
help(np.random.multivariate_normal)

Help on built-in function multivariate_normal:

multivariate_normal(...) method of numpy.random.mtrand.RandomState instance
    multivariate_normal(mean, cov, size=None, check_valid='warn', tol=1e-8)
    
    Draw random samples from a multivariate normal distribution.
    
    The multivariate normal, multinormal or Gaussian distribution is a
    generalization of the one-dimensional normal distribution to higher
    dimensions.  Such a distribution is specified by its mean and
    covariance matrix.  These parameters are analogous to the mean
    (average or "center") and variance (standard deviation, or "width,"
    squared) of the one-dimensional normal distribution.
    
    .. note::
        New code should use the ``multivariate_normal`` method of a ``default_rng()``
        instance instead; please see the :ref:`random-quick-start`.
    
    Parameters
    ----------
    mean : 1-D array_like, of length N
        Mean of the N-dimensional distribution.
    cov : 2-D array_like,

In [159]:
n = 1
d = 3
p = 10

sample_grads = .1* np.random.multivariate_normal(np.zeros(d), np.identity(d),p)

dg_M = np.zeros((n,d ,p))
dg_M[0] = rearrange(sample_grads, 'p d -> d p')
df_M = np.zeros((n,d ,d))
df_M[0] = np.identity(d)

In [160]:
# sample_grads = .1* np.random.multivariate_normal(np.zeros(2), np.identity(2),10)
# n = 2
# d = 2
# p = 10
# dg_M = np.zeros((n,d ,p))
# dg_M[0] = rearrange(sample_grads, 'p d -> d p')
# sample_grads = .1* np.random.multivariate_normal(np.zeros(2), np.identity(2),10)
# dg_M[1] = rearrange(sample_grads, 'p d -> d p')

# df_M = np.zeros((n,d ,d))
# df_M[0] = np.identity(2)
# df_M[1] = np.identity(2)

In [163]:
gl_itermax = 100000
tol = 1e-16
learning_rate = 10000

In [164]:
GGL = GradientGroupLasso(dg_M, df_M, np.asarray([.00001]), 0., gl_itermax,learning_rate, tol, beta0_npm= None)
GGL.fit()

In [165]:
# shouldnt we just keep the longest most orthogonal pair?  No.

In [166]:
# it looks like maybe it works...
# before we could see that the dual problem minimizer was actually not the cardinality 2 solution.

In [167]:
GGL.fit_[0]['beta'].shape

(1, 10, 3)

In [168]:
import numpy as np
np.einsum('i p d, i e p -> i d e',GGL.fit_[0]['beta'], dg_M)

array([[[ 9.99957330e-01,  8.88559915e-06, -1.62339409e-05],
        [ 8.88533368e-06,  9.99959932e-01,  1.51365494e-05],
        [-1.62332078e-05,  1.51365592e-05,  9.99953326e-01]]])

In [169]:
GGL._L1penalty(GGL.fit_[0]['beta'])

12.940365223953933

In [170]:
GGL.fit_[0]['beta']

array([[[ 1.77091266,  1.42529816, -2.37471106],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 1.25847749, -1.90490518,  0.860803  ],
        [ 0.31773416,  1.8804253 ,  1.21070731],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 3.60199294, -1.29040027,  3.14699365],
        [ 0.        ,  0.        ,  0.        ]]])

In [171]:
from itertools import combinations
subsets =  list(combinations(list(range(10)), 3))
asdf = []
for subset in subsets:
    asdf.append(GGL._L1penalty(np.linalg.inv(dg_M[:,:,subset])))

In [172]:
??GGL._L1penalty

In [173]:
def argmin_indices(lst):
    min_value = min(lst)  # Find the minimum value in the list
    return [i for i, x in enumerate(lst) if x == min_value]  # List indices where the element is the minimum value

argmin_indices(asdf)

[28]

In [175]:
subsets[28]

(0, 5, 8)

In [148]:
from itertools import combinations
subsets =  list(combinations(list(range(10)), 3))
asdf = []
for subset in subsets:
    asdf.append(GGL._L1penalty(dg_M[:,:,subset]))

In [149]:
min(asdf)

0.2634408767807169

In [150]:
from itertools import combinations
subsets =  list(combinations(list(range(10)), 3))
asdf = []
for subset in subsets:
    asdf.append(GGL._L1penalty(np.transpose(np.linalg.inv(dg_M[:,:,subset]))))

In [152]:
min(asdf)

14.193900509597373

In [ ]:
beta0_npm = GGL.fit_[-1]['beta']
coeffs[probe_init_low] = GGL.fit_[-1]['beta']
combined_norms[probe_init_low] = np.sqrt((np.linalg.norm(coeffs[probe_init_low], axis = 0)**2).sum(axis = 1))


You can express any $x_j = \gamma^T x_{[d]}$ where $x_{[d]} \in R^{d \times d}$.

Can we express certain $x_j$ more cheaply i.e. with $\|\gamma\|_2 < 1$? No, not always (think about |<).

But even in that case the ultimate selection is d-sparse.

